In [352]:
import napari
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.types import PhysicalPixelSizes
from skimage.measure import label

# 2D
generate data

In [353]:
# Initialize an empty 1000x1000 array of uint16 type
array_2d = np.zeros((1000, 1000), dtype=np.uint16)

# Define square sizes (areas 100, 81, 900, 1000)
square_sizes = [100, 81, 900, 1000]
square_sides = [int(np.sqrt(size)) for size in square_sizes]

# Define top-left coordinates for the squares (ensuring no overlap)
# Manually chosen positions to avoid overlap
positions = [(50, 50), (200, 200), (500, 500), (800, 50)]

# Fill in the squares with arbitrary non-zero values
for i, (side, (x, y)) in enumerate(zip(square_sides, positions)):
    array_2d[x:x+side, y:y+side] = (i + 1) * 1000  # Assign values 1000, 2000, 3000, 4000

# Initialize another empty 1000x1000 array of uint16 type
small_squares_array = np.zeros((1000, 1000), dtype=np.uint16)

# Side length of the small squares (area 4 means side length 2)
small_square_side = 2

# Function to check if a square is inside the bounds of a large square
def is_in_large_square(x, y, large_squares):
    for lx, ly, side in large_squares:
        if lx <= x < lx + side and ly <= y < ly + side:
            return True
    return False

# Large squares' positions and sides from the previous array
large_squares_info = [(50, 50, square_sides[0]), (200, 200, square_sides[1]), 
                      (500, 500, square_sides[2]), (800, 50, square_sides[3])]

# Generate 50 small squares inside the large squares
inside_count = 0
while inside_count < 50:
    # Randomly choose a large square
    lx, ly, side = large_squares_info[np.random.randint(0, 4)]
    # Random position within the large square
    x = np.random.randint(lx, lx + side - small_square_side - 1)
    y = np.random.randint(ly, ly + side - small_square_side - 1)
    # Place the square if there's no overlap
    if np.all(small_squares_array[x-1:x+small_square_side+1, y-1:y+small_square_side+1] == 0):
        small_squares_array[x:x+small_square_side, y:y+small_square_side] = 5000  # Arbitrary value
        inside_count += 1

# Generate 50 small squares outside the large squares
outside_count = 0
while outside_count < 50:
    # Random position anywhere in the array
    x = np.random.randint(0, 1000 - small_square_side - 1)
    y = np.random.randint(0, 1000 - small_square_side - 1)
    # Place the square if there's no overlap with large squares or previously placed small squares
    if np.all(small_squares_array[x-1:x+small_square_side+1, y-1:y+small_square_side+1] == 0) and not is_in_large_square(x, y, large_squares_info):
        small_squares_array[x:x+small_square_side, y:y+small_square_side] = 1000
        outside_count += 1

# Initialize a new empty 1000x1000 array for the overlapping squares
overlapping_squares_array = np.zeros((1000, 1000), dtype=np.uint16)

# Define new positions for the overlapping squares, ensuring partial overlap with the previous large squares
overlapping_positions = [(49, 49), (195, 195), (490, 490), (790, 40)]

# Fill in the overlapping squares with arbitrary non-zero values
for i, (side, (x, y)) in enumerate(zip(square_sides, overlapping_positions)):
    overlapping_squares_array[x:x+side, y:y+side] = (i + 5) * 1000  # Assign different values from the previous image

In [354]:
def translate_with_periodic_boundary(array, shift_x, shift_y):
    """
    Translates a 2D NumPy array by (shift_x, shift_y) with periodic boundary conditions.

    Parameters:
    array (np.ndarray): Input 2D array to translate.
    shift_x (int): Amount to shift in the x-direction (rows).
    shift_y (int): Amount to shift in the y-direction (columns).

    Returns:
    np.ndarray: The translated array with periodic boundary conditions.
    """
    # Use numpy's roll function to handle periodic boundary conditions
    translated_array = np.roll(array, shift=(shift_x, shift_y), axis=(0, 1))
    
    return translated_array


translate data in x and y for time-series

In [355]:
intensity_arrays = [array_2d,small_squares_array,overlapping_squares_array]
t1 = np.stack(intensity_arrays)[:,np.newaxis,:,:]
t2 = np.stack([translate_with_periodic_boundary(a,10,10) for a in intensity_arrays])[:,np.newaxis,:,:]
t3 = np.stack([translate_with_periodic_boundary(a,20,20) for a in intensity_arrays])[:,np.newaxis,:,:]
intensity_image = AICSImage(np.stack([t1,t2,t3]),channel_names=["Channel1","Channel2","Channel3"],physical_pixel_sizes=PhysicalPixelSizes(Z=1,Y=1,X=1))

visualise data

In [356]:
viewer = napari.Viewer()

In [357]:
viewer.add_image(intensity_image.get_image_data('TCYX',Z=0),channel_axis=1,name=intensity_image.channel_names)

[<Image layer 'Channel1' at 0x4797bb340>,
 <Image layer 'Channel2' at 0x4797b5270>,
 <Image layer 'Channel3' at 0x4369cffd0>]

In [358]:
intensity_image.physical_pixel_sizes

PhysicalPixelSizes(Z=1, Y=1, X=1)

segment all channels

In [359]:
ctyx_labels = np.stack([np.stack([label(intensity_image.get_image_data('YX',T=t,C=ch,Z=0)>0) for t in range(intensity_image.dims.T)]) for ch in range(intensity_image.dims.C)])
tczyx_labels = np.swapaxes(ctyx_labels,0,1)[:,:,np.newaxis,:,:]

In [360]:
label_image = AICSImage(tcyx_labels.astype(np.int32),
                        channel_names=["Object1","Object2","Object3"],
                        physical_pixel_sizes=PhysicalPixelSizes(Z=1,Y=1,X=1))

visualise segmentation

In [361]:
for i in range(label_image.dims.C):
    viewer.add_labels(label_image.get_image_data('TYX',Z=0,C=i),name=label_image.channel_names[i])

In [362]:
intensity_image.save("synthetic_intensity_image_TYX.tiff")
label_image.save("synthetic_label_image_TYX.tiff")

# 3D
generate data

In [363]:
# Step 1: Generate the first 3D array with non-overlapping cubes
array_3d = np.zeros((200, 200, 200), dtype=np.uint16)

# Define cube volumes (1000, 729, 27000, 100000)
cube_volumes = [1000, 729, 27000, 100000]
cube_sides = [int(np.cbrt(volume)) for volume in cube_volumes]

# Define top-left-front coordinates for the cubes (ensuring no overlap)
positions_3d = [(5, 5, 5), (20, 20, 20), (50, 50, 50), (100, 5, 5)]

# Fill in the cubes with arbitrary non-zero values
for i, (side, (x, y, z)) in enumerate(zip(cube_sides, positions_3d)):
    array_3d[x:x+side, y:y+side, z:z+side] = (i + 1) * 1000  # Assign values 1000, 2000, 3000, 4000

# Step 2: Generate the second 3D array with small cubes (volume 8)
small_cubes_array_3d = np.zeros((200, 200, 200), dtype=np.uint16)

# Small cube side (volume 8 means side length 2)
small_cube_side = 2

# Function to check if a cube is inside the bounds of a large cube
def is_in_large_cube(x, y, z, large_cubes):
    for lx, ly, lz, side in large_cubes:
        if lx <= x < lx + side and ly <= y < ly + side and lz <= z < lz + side:
            return True
    return False

# Large cubes' positions and sides from the first array
large_cubes_info = [(5, 5, 5, cube_sides[0]), (20, 20, 20, cube_sides[1]), 
                    (50, 50, 50, cube_sides[2]), (80, 5, 5, cube_sides[3])]

# Generate 50 small cubes inside the large cubes
inside_count = 0
while inside_count < 50:
    # Randomly choose a large cube
    lx, ly, lz, side = large_cubes_info[np.random.randint(0, 4)]
    # Random position within the large cube
    x = np.random.randint(lx, lx + side - small_cube_side - 1)
    y = np.random.randint(ly, ly + side - small_cube_side - 1)
    z = np.random.randint(lz, lz + side - small_cube_side - 1)
    # Place the cube if there's no overlap
    if np.all(small_cubes_array_3d[x-1:x+small_cube_side+1, y-1:y+small_cube_side+1, z-1:z+small_cube_side+1] == 0):
        small_cubes_array_3d[x:x+small_cube_side, y:y+small_cube_side, z:z+small_cube_side] = 5000  # Arbitrary value
        inside_count += 1

# Generate 50 small cubes outside the large cubes
outside_count = 0
while outside_count < 50:
    # Random position anywhere in the array
    x = np.random.randint(0, 200 - small_cube_side - 1)
    y = np.random.randint(0, 200 - small_cube_side - 1)
    z = np.random.randint(0, 200 - small_cube_side - 1)
    # Place the cube if there's no overlap with large cubes or previously placed small cubes
    if np.all(small_cubes_array_3d[x-1:x+small_cube_side+1, y-1:y+small_cube_side+1, z-1:z+small_cube_side+1] == 0) and not is_in_large_cube(x, y, z, large_cubes_info):
        small_cubes_array_3d[x:x+small_cube_side, y:y+small_cube_side, z:z+small_cube_side] = 6000  # Arbitrary value
        outside_count += 1

# Step 3: Generate the third 3D array with overlapping cubes
overlapping_cubes_array_3d = np.zeros((200, 200, 200), dtype=np.uint16)

# Define new positions for the overlapping cubes, ensuring partial overlap with the previous large cubes
overlapping_positions_3d = [(3, 3, 3), (18, 18, 18), (48, 48, 48), (98, 3, 3)]

# Fill in the overlapping cubes with arbitrary non-zero values
for i, (side, (x, y, z)) in enumerate(zip(cube_sides, overlapping_positions_3d)):
    overlapping_cubes_array_3d[x:x+side, y:y+side, z:z+side] = (i + 5) * 1000  # Assign different values from the previous cubes

In [364]:
def translate_with_periodic_boundary_3d(array, shift_x, shift_y, shift_z):
    """
    Translates a 3D NumPy array by (shift_x, shift_y, shift_z) with periodic boundary conditions.

    Parameters:
    array (np.ndarray): Input 3D array to translate.
    shift_x (int): Amount to shift in the x-direction (depth).
    shift_y (int): Amount to shift in the y-direction (rows).
    shift_z (int): Amount to shift in the z-direction (columns).

    Returns:
    np.ndarray: The translated array with periodic boundary conditions.
    """
    # Use numpy's roll function to handle periodic boundary conditions for 3D arrays
    translated_array = np.roll(array, shift=(shift_x, shift_y, shift_z), axis=(0, 1, 2))
    
    return translated_array

In [365]:
intensity_arrays_3d = [array_3d,small_cubes_array_3d,overlapping_cubes_array_3d]
t1 = np.stack(intensity_arrays_3d)
t2 = np.stack([translate_with_periodic_boundary_3d(a,2,2,2) for a in intensity_arrays_3d])
t3 = np.stack([translate_with_periodic_boundary_3d(a,4,4,4) for a in intensity_arrays_3d])
t4 = np.stack([translate_with_periodic_boundary_3d(a,6,6,6) for a in intensity_arrays_3d])
intensity_image_3d = AICSImage(np.stack([t1,t2,t3,t4],axis=0),
                               channel_names=["Channel1","Channel2","Channel3"],
                               physical_pixel_sizes=PhysicalPixelSizes(Z=1,Y=1,X=1))

In [366]:
intensity_image_3d.dims

<Dimensions [T: 4, C: 3, Z: 200, Y: 200, X: 200]>

In [367]:
viewer_3d = napari.Viewer()

In [368]:
viewer_3d.add_image(intensity_image_3d.get_image_data('TCZYX'),channel_axis=1,name=intensity_image_3d.channel_names)

[<Image layer 'Channel1' at 0x46e1fca60>,
 <Image layer 'Channel2' at 0x38d4c7f10>,
 <Image layer 'Channel3' at 0x38d3ee0e0>]

In [369]:
ctzyx_labels = np.stack([np.stack([label(intensity_image_3d.get_image_data('ZYX',T=t,C=ch)>0) for t in range(intensity_image_3d.dims.T)]) for ch in range(intensity_image_3d.dims.C)])
tczyx_labels = np.swapaxes(ctzyx_labels,0,1)

In [370]:
label_image_3d = AICSImage(tczyx_labels.astype(np.int32),
                           channel_names=["Object1","Object2","Object3"],
                           physical_pixel_sizes=PhysicalPixelSizes(Z=1,Y=1,X=1))

visualise segmentation

In [371]:
for i in range(label_image_3d.dims.C):
    viewer_3d.add_labels(label_image_3d.get_image_data('TZYX',C=i),name=label_image_3d.channel_names[i])

In [372]:
intensity_image_3d.save("synthetic_intensity_image_TZYX.tiff")
label_image_3d.save("synthetic_label_image_TZYX.tiff")